In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pywt
import numpy as np
import pandas as pd
from numpy import argmax
import matplotlib.pyplot as plt
from pandas import DataFrame

import graphviz
import xgboost as xgb
from xgboost import plot_tree
from scipy import signal

import sklearn
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bledataset/BLE_rssi_after_FFT_Filter_Grid_1.5m_2.5mAnalysis.csv
/kaggle/input/bledataset/BLE_rssi_after_Kalman_Filter_Grid_1.5m_1.25m.csv
/kaggle/input/bledataset/BLE_rssi_after_FFT_Filter_Grid_1.5m_2.5m.csv
/kaggle/input/bledataset/BLE_rssi.csv
/kaggle/input/bledataset/BLE_rssi_Before_Filter_Grid_1.5m_1.25m.csv
/kaggle/input/bledataset/BLE_rssi_after_FFT_Filter_Grid_1.5m_1.25m.csv
/kaggle/input/bledataset/BLE_rssi_after_Gray_Filter_Grid_1.5m_1.25m.csv
/kaggle/input/bledataset/BLE_rssi_Before_Filter_Grid_1.5m_2.5m.csv


In [2]:
data = pd.read_csv('../input/bledataset/BLE_rssi_after_FFT_Filter_Grid_1.5m_1.25m.csv')

data = data.drop(['date'],axis=1)

my_dict = {}
l = []

"""Instructions:
In following we will apply wavelet transformationm.
The transformation is a two dimential matix.
I took first column as a transformed data.
That provides better result.
I then do it for multi times.
That is, every time, I took first column of the transformed data,
And then I re-apply wavelet on the extracted column data.
How many times do we want to perform that wavelet transform,
that I controlled by t."""

t = 3
widths = np.arange(1, len(data)+1)

for k in range(1,t):
    for i in range(1,6):
        cwtmatr = signal.cwt(data.iloc[:,i], signal.ricker, widths)
        (cA, cD) = pywt.dwt([1,2,3,4,5,6], 'db1')

        cwtmatr=cwtmatr.astype(int)
        data.iloc[:,i]=cwtmatr[:,0]

X = data.iloc[:,1:]
y = data.iloc[:,0]

#Encoder
le = LabelEncoder()
y = le.fit_transform(y)

trainX, testX, trainY, testY = train_test_split(X, y, test_size= 0.1, random_state=0)

### XG Boost

In [3]:
xgb_classifier = xgb.XGBClassifier(max_depth=26, learning_rate=0.2, n_estimators=220, silent=True, objective='multi:softprob', booster='gbtree')
xgb_classifier.fit(trainX, trainY)

"""plot_tree(xgb_classifier,  num_trees=4)
fig = plt.gcf()
fig.set_size_inches(150, 100)
fig.savefig('tree.png')
plt.show()"""

predY_XGB = xgb_classifier.predict(testX)

modelName = "XGB"
accuracy = round(accuracy_score(testY, predY_XGB),4)*100 
precision = round(precision_score(testY, predY_XGB, average='micro'),4)*100 
recall = round(recall_score(testY, predY_XGB, average='micro'),4)*100 
f1 = round(f1_score(testY, predY_XGB, average='micro'),4)*100

l.append([modelName, accuracy, precision, recall, f1])

#my_dict['Mean Absolute Error'] = metrics.mean_absolute_error(testY, predY_XGB)
#my_dict['Mean Squared Error'] = metrics.mean_squared_error(testY, predY_XGB)
#my_dict['Root Mean Squared Error'] = np.sqrt(metrics.mean_squared_error(testY, predY_XGB))

[19:45:36] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




### Random Forest

In [4]:
regressor = sklearn.ensemble.RandomForestRegressor(n_estimators = 220, random_state = 2)
regressor.fit(trainX,trainY)

predY_RF = regressor.predict(testX)

yy=[]
for i in range(0,len(predY_RF)):
    yy.append(int(predY_RF[i]))
predY_RF = yy

modelName = "Random Forest"
accuracy = round(accuracy_score(testY, predY_RF),4)*100 
precision = round(precision_score(testY, predY_RF, average='micro'),4)*100 
recall = round(recall_score(testY, predY_RF, average='micro'),4)*100 
f1 = round(f1_score(testY, predY_RF, average='micro'),4)*100
l.append([modelName, accuracy, precision, recall, f1])

### GaussianNB

In [5]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(trainX, trainY)
predY=gnb.predict(testX)

modelName = "GaussianNB"
accuracy = round(accuracy_score(testY, predY),4)*100 
precision = round(precision_score(testY, predY, average='micro'),4)*100 
recall = round(recall_score(testY, predY, average='micro'),4)*100 
f1 = round(f1_score(testY, predY, average='micro'),4)*100
l.append([modelName, accuracy, precision, recall, f1])

In [6]:
df = DataFrame (l, columns=[ 'Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
df

,Model,Accuracy,Precision,Recall,F1-Score
0,XGB,97.81,97.81,97.81,97.81
1,Random Forest,88.42,88.42,88.42,88.42
2,GaussianNB,60.88,60.88,60.88,60.88


# 2nd Part

In [7]:
import pandas as pd
import numpy as np
from numpy import argmax

from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
#from keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
#from tensorflow.keras.utils import to_categorical
from scipy import signal

In [8]:
data = pd.read_csv('../input/bledataset/BLE_rssi_after_FFT_Filter_Grid_1.5m_2.5m.csv')

data=data.drop(['date'],axis=1)
t=2
widths = np.arange(1, len(data)+1)

In [9]:
#"""Activate thi section when you want to apply wavelet transform
for k in range(1,t):
    for i in range(1,6):
        cwtmatr = signal.cwt(data.iloc[:,i], signal.ricker, widths)
        cwtmatr=cwtmatr.astype(int)
        data.iloc[:,i]=cwtmatr[:,0]
#"""

In [10]:
X = data.iloc[:,1:]
y = data.iloc[:,0]
trainX, testX, trainY, testY = train_test_split(X, y, test_size= 0.1, random_state=42)
orgtestY = testY

#Learning parameter initialization
windowX=5
windowY=5
n_classes = len(data.iloc[:,0].unique())
kernelCNN_Dim=(3,3)
epochs=200
batch_size=128

### SVC

#### SVC Poly

In [14]:
svcPoly = SVC(kernel='poly', degree=3)
svcPoly.fit(trainX, trainY)
predPolyX = svcPoly.predict(testX)

modelName = "SVC Poly"
accuracy = round(accuracy_score(testY, predPolyX),4)*100 
precision = round(precision_score(testY, predPolyX, average='micro'),4)*100 
recall = round(recall_score(testY, predPolyX, average='micro'),4)*100 
f1 = round(f1_score(testY, predPolyX, average='micro'),4)*100
l.append([modelName, accuracy, precision, recall, f1])

#### SVC Sigmoid

In [15]:
svcSigmoid = SVC(kernel='sigmoid')
svcSigmoid.fit(trainX, trainY)
predSigmoidX = svcSigmoid.predict(testX)

modelName = "SVC Sigmoid"
accuracy = round(accuracy_score(testY, predSigmoidX),4)*100 
precision = round(precision_score(testY, predSigmoidX, average='micro'),4)*100 
recall = round(recall_score(testY, predSigmoidX, average='micro'),4)*100 
f1 = round(f1_score(testY, predSigmoidX, average='micro'),4)*100
l.append([modelName, accuracy, precision, recall, f1])

### One-hot encoding

In [18]:
targetY = trainY
targetY=targetY.append(testY)
Y_target=pd.get_dummies(targetY)

totalTrainingRow = int(len(data) * 0.9)

Y_train=Y_target.iloc[0:totalTrainingRow]
Y_test=Y_target.iloc[totalTrainingRow:]

### DNN 

In [22]:
import tensorflow_addons as tfa
import tensorflow as tf

In [23]:
model = Sequential()
model.add(Dense(50, input_shape=(trainX.shape[1],), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(Y_train.shape[1], activation='softmax'))
model.summary()

metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(num_classes = Y_train.shape[1]), 
               tf.keras.metrics.AUC(), tfa.metrics.CohenKappa(num_classes = Y_train.shape[1])]

model.compile(loss='categorical_crossentropy', metrics = metrics, optimizer='adam')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 50)                300       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_5 (Dense)              (None, 55)                5555      
Total params: 10,955
Trainable params: 10,955
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(trainX, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(testX, Y_test))

2022-07-04 20:24:55.023258: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
45/45 [==============================] - 4s 27ms/step - loss: 242.9840 - accuracy: 0.0512 - precision: 0.0501 - recall: 0.0482 - f1_score: 0.0512 - auc: 0.5297 - cohen_kappa: 0.0294 - val_loss: 71.0952 - val_accuracy: 0.0798 - val_precision: 0.0783 - val_recall: 0.0751 - val_f1_score: 0.0464 - val_auc: 0.5647 - val_cohen_kappa: 0.0679
Epoch 2/200
45/45 [==============================] - 0s 6ms/step - loss: 48.6252 - accuracy: 0.1182 - precision: 0.1114 - recall: 0.1015 - f1_score: 0.1246 - auc: 0.5906 - cohen_kappa: 0.0986 - val_loss: 30.2881 - val_accuracy: 0.2019 - val_precision: 0.2090 - val_recall: 0.1815 - val_f1_score: 0.1777 - val_auc: 0.6395 - val_cohen_kappa: 0.1850
Epoch 3/200
45/45 [==============================] - 0s 6ms/step - loss: 24.2156 - accuracy: 0.1726 - precision: 0.1783 - recall: 0.1461 - f1_score: 0.1836 - auc: 0.6266 - cohen_kappa: 0.1533 - val_loss: 26.1288 - val_accuracy: 0.2316 - val_precision: 0.2800 - val_recall: 0.2081 - val_f1_score: 0.2200 -

In [37]:
#modelPred = argmax(model.predict(testX),axis=1)
modelName = "DNN"
accuracy = .4225 * 100 
precision = .9754 * 100 
recall = .3099* 100 
f1 = .3632 * 100 
l.append([modelName, accuracy, precision, recall, f1])

### CNN

In [31]:
x_test = testX
y_test = Y_test

# implement code to predict using CNN

#convert 1D pattern to 1Dx1D matrix
def OnedToTwoDClone(pattern):
    Array2D = [pattern] + [pattern for i in pattern]
    return np.vstack(tuple(Array2D))

#convert 1D pattern to 1Dxrows matrix
def OneToTwoDClone(pattern, rows):
    return np.reshape(np.tile(pattern, rows), len(pattern)*rows)

#convert whole dataset to size of columns x rows and flatten them 
#then generate the dataset
trainX = np.tile(trainX, np.shape(trainX)[1])
testX = np.tile(testX, np.shape(testX)[1])

print(np.shape(trainX))
print(np.shape(testX))

trainX = trainX.astype('float32')
testX = testX.astype('float32')
maxVal = np.max(np.max(data.iloc[:,1:-1]))

trainX /= maxVal
testX /= maxVal
# Change the data dimensionality according to your data here
trainX = trainX.reshape(trainX.shape[0], windowX, windowY, 1)
testX = testX.reshape(testX.shape[0], windowX, windowY, 1)

(5743, 25)
(639, 25)


In [32]:
modelCNN = Sequential()
modelCNN.add(Conv2D(150, kernel_size=kernelCNN_Dim, strides=(1,1), padding='valid', activation='relu', input_shape=(windowX, windowY,1)))
modelCNN.add(Conv2D(200, kernel_size=kernelCNN_Dim, strides=(1,1), padding='valid', activation='relu'))
modelCNN.add(Flatten())
modelCNN.add(Dense(100, activation='relu'))
modelCNN.add(Dense(150, activation='relu'))
modelCNN.add(Dense(n_classes, activation='softmax'))#Number of classes 55
metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(num_classes = n_classes), 
               tf.keras.metrics.AUC(), tfa.metrics.CohenKappa(num_classes = n_classes)]

modelCNN.compile(loss='categorical_crossentropy', metrics = metrics, optimizer='adam')
modelCNN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 3, 150)         1500      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 200)         270200    
_________________________________________________________________
flatten (Flatten)            (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_7 (Dense)              (None, 150)               15150     
_________________________________________________________________
dense_8 (Dense)              (None, 55)                8305      
Total params: 315,255
Trainable params: 315,255
Non-trainable params: 0
________________________________________________

In [33]:
modelCNN.fit(trainX, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(testX, Y_test))

Epoch 1/200
45/45 [==============================] - 4s 43ms/step - loss: 3.6693 - accuracy: 0.0881 - precision_1: 0.5938 - recall_1: 0.0033 - f1_score: 0.0370 - auc_1: 0.7099 - cohen_kappa: 0.0440 - val_loss: 3.3048 - val_accuracy: 0.0720 - val_precision_1: 0.8125 - val_recall_1: 0.0203 - val_f1_score: 0.0213 - val_auc_1: 0.8001 - val_cohen_kappa: 0.0434
Epoch 2/200
45/45 [==============================] - 1s 21ms/step - loss: 3.1780 - accuracy: 0.1391 - precision_1: 0.6654 - recall_1: 0.0301 - f1_score: 0.0935 - auc_1: 0.8211 - cohen_kappa: 0.0993 - val_loss: 2.8807 - val_accuracy: 0.2332 - val_precision_1: 0.8462 - val_recall_1: 0.0861 - val_f1_score: 0.1587 - val_auc_1: 0.8583 - val_cohen_kappa: 0.2081
Epoch 3/200
45/45 [==============================] - 1s 20ms/step - loss: 2.8052 - accuracy: 0.2204 - precision_1: 0.7865 - recall_1: 0.0834 - f1_score: 0.1939 - auc_1: 0.8681 - cohen_kappa: 0.1914 - val_loss: 2.5808 - val_accuracy: 0.2629 - val_precision_1: 0.7849 - val_recall_1: 0.

In [38]:
#modelCNNPred = argmax(modelCNN.predict(testX),axis=1)

In [39]:
modelName = "CNN"
accuracy = 0.5396 * 100 
precision = 0.8584 * 100 
recall = 0.4667 * 100 
f1 = 0.4963 * 100 
l.append([modelName, accuracy, precision, recall, f1])

In [47]:
c = [['XGB', 97.81, 97.81, 97.81, 97.81], ['Random Forest', 88.42, 88.42, 88.42, 88.42], ['GaussianNB', 60.88, 60.88, 60.88, 60.88], ['SVC Poly',  11.110000000000001,11.110000000000001, 11.110000000000001, 11.110000000000001],['SVC Sigmoid', 7.82, 7.82, 7.82, 7.82], ['DNN', 42.25, 97.54, 30.990000000000002, 36.32], ['CNN', 53.959999999999994, 85.84, 46.67, 49.63]]

In [48]:
df = DataFrame (c, columns=[ 'Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
df

,Model,Accuracy,Precision,Recall,F1-Score
0,XGB,97.81,97.81,97.81,97.81
1,Random Forest,88.42,88.42,88.42,88.42
2,GaussianNB,60.88,60.88,60.88,60.88
3,SVC Poly,11.11,11.11,11.11,11.11
4,SVC Sigmoid,7.82,7.82,7.82,7.82
5,DNN,42.25,97.54,30.99,36.32
6,CNN,53.96,85.84,46.67,49.63


## Stack of Traditional Models 

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

def ml_models_for_unigram_tfidf(): 
    """
    This function consists the models defination for Unigram Features
    
    Retuns:
        ml_models: list of models
        model_names: list of model_names
    
    """
    
    lr_model = LogisticRegression(random_state = 123)
    dt_model = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    rf_model = RandomForestClassifier(n_estimators=100, criterion ='entropy', random_state = 0)
    mnb_model = MultinomialNB(alpha=0.15)
    knn_model = KNeighborsClassifier(n_neighbors=3, metric = 'minkowski')
    #lsvm_model = SVC(kernel = 'linear',C = 0.2, probability=True, random_state = 0)
    ksvm_model = SVC(C= 1000,kernel = 'rbf',probability=True, gamma = 0.00015, random_state = 0)
    sgd_model = SGDClassifier(loss ='log',penalty='l2', max_iter=5)
    xgb_classifier = xgb.XGBClassifier(max_depth=26, learning_rate=0.2, n_estimators=220, silent=True, objective='multi:softprob', booster='gbtree')
    #model_names = ['Logistic Regression','Decision Tree','Random Forest','Naive Bayes','KNN','Linear SVM','Kernel SVM','SGD']
    model_names = ['Logistic Regression','Decision Tree','Random Forest','KNN','Kernel SVM','SGD', 'XGB']
    # Create list of models
    #ml_models = [lr_model,dt_model,rf_model,mnb_model,knn_model,lsvm_model,ksvm_model,sgd_model]
    ml_models = [lr_model,dt_model,rf_model,knn_model,ksvm_model,sgd_model, xgb_classifier]
    return ml_models,model_names    


def ml_models_for_trigram_tfidf(): 
    """
    This function consists the models defination for Tri-gram Features
    
    Retuns:
        ml_models: list of models
        model_names: list of model_names
    
    """
    
    lr_model = LogisticRegression(random_state = 123)
    dt_model = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    rf_model = RandomForestClassifier(n_estimators=100, criterion ='entropy', random_state = 0)
    mnb_model = MultinomialNB(alpha=0.001)
    knn_model = KNeighborsClassifier(n_neighbors=2, metric = 'minkowski')
    lsvm_model = SVC(kernel = 'linear',C = 0.3, probability=True, random_state = 0)
    ksvm_model = SVC(C= 1000,kernel = 'rbf',probability=True, gamma = 0.0002, random_state = 0)
    sgd_model = SGDClassifier(loss = 'log',penalty='l2', max_iter=5)
    xgb_classifier = xgb.XGBClassifier(max_depth=26, learning_rate=0.2, n_estimators=220, silent=True, objective='multi:softprob', booster='gbtree')
    model_names = ['Logistic Regression','Decision Tree','Random Forest','KNN','Kernel SVM','SGD', 'XGB']
    
    #create a list of models
    ml_models = [lr_model,dt_model,rf_model,knn_model,ksvm_model,sgd_model, xgb_classifier]
    return ml_models,model_names  



def model_performace(model,X_train,X_test,y_train,y_test):
    """
    This function will return the performance parameter values of each ML models.
    Performance parameters are Accuracy, F1-Score, Precision, Recall.
    
    Args:
        model: a ML model instance
        X_train: training feature vector (sparse matrix)
        X_test : testing feature vector (sparse matrix)
        y_train: training encoded labels (array) 
        y_test : testing encoded labels (array) 
        
    Returns:
        my_dict: a dictionary of all the parameters for each models
    """
    my_dict = {}
    model.fit(X_train,y_train)
    # Prediction
    pred_y = model.predict(X_test)
    my_dict['Accuracy'] = round(accuracy_score(y_test, pred_y),4)*100 
    my_dict['Precision'] = round(precision_score(y_test, pred_y, average='micro'),4)*100 
    my_dict['Recall'] = round(recall_score(y_test, pred_y, average='micro'),4)*100 
    my_dict['F1 Score'] = round(f1_score(y_test, pred_y, average='micro'),4)*100 
    print("Model => {} Accuracy: {} Precision: {} Recall {} F1-Score {}".format(model, my_dict['Accuracy'], my_dict['Precision'], my_dict['Recall'], my_dict['F1 Score'] ))
    return my_dict  

            
def performance_table(performance_dict):
    """
    This function will create a dataframe of all the performance parameters.
    
    Args:
        performance_dict: a dictionary of all the parameters for each models
        
    Returns:
        performance_df: a dataframe
    """

    acc_list = []
    pr_list = []
    re_list = []
    f1_list = []
    for i in performance_dict.keys():
        acc_list.append(performance_dict[i]['Accuracy'])
        pr_list.append(performance_dict[i]['Precision'])
        re_list.append(performance_dict[i]['Recall'])
        f1_list.append(performance_dict[i]['F1 Score'])

    # Create a dataframe
    performance_df = pd.DataFrame({'Accuracy':acc_list,'Precision':pr_list,
                                   'Recall':re_list,'F1 Score':f1_list},
                                  index =['LR','DT','RF','MNB','KNN','RBF SVM','SGD'])
    return performance_df

In [47]:
## classifiers defination
ml_models,model_names = ml_models_for_unigram_tfidf() 

In [48]:
# call model accuracy function and save the metrices into a dictionary
accuracy = {f'{model_names[i]}':model_performace(model,trainX, testX, trainY, testY) for i,model in enumerate(ml_models)}

# Save the performance parameter into json file
with open('ml_performance_unigram.json', 'w') as f:
    json.dump(accuracy, f)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Model => LogisticRegression(random_state=123) Accuracy: 9.700000000000001 Precision: 9.700000000000001 Recall 9.700000000000001 F1-Score 9.700000000000001
Model => DecisionTreeClassifier(criterion='entropy', random_state=0) Accuracy: 97.81 Precision: 97.81 Recall 97.81 F1-Score 97.81
Model => RandomForestClassifier(criterion='entropy', random_state=0) Accuracy: 98.59 Precision: 98.59 Recall 98.59 F1-Score 98.59
Model => KNeighborsClassifier(n_neighbors=3) Accuracy: 97.65 Precision: 97.65 Recall 97.65 F1-Score 97.65
Model => SVC(C=1000, gamma=0.00015, probability=True, random_state=0) Accuracy: 98.11999999999999 Precision: 98.11999999999999 Recall 98.11999999999999 F1-Score 98.11999999999999


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


Model => SGDClassifier(loss='log', max_iter=5) Accuracy: 2.6599999999999997 Precision: 2.6599999999999997 Recall 2.6599999999999997 F1-Score 2.6599999999999997
[11:17:23] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Model => XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=26, max_leaves=0,

In [49]:
# Load the json file
accuracy = json.load(open('ml_performance_unigram.json'))
table = performance_table(accuracy)
table

,Accuracy,Precision,Recall,F1 Score
LR,9.70,9.70,9.70,9.70
DT,97.81,97.81,97.81,97.81
RF,98.59,98.59,98.59,98.59
MNB,97.65,97.65,97.65,97.65
KNN,98.12,98.12,98.12,98.12
RBF SVM,2.66,2.66,2.66,2.66
SGD,97.81,97.81,97.81,97.81


In [50]:
print(f"Highest Accuracy achieved by {table.Accuracy.idxmax(axis = 0)} at = {max(table.Accuracy)}")
print(f"Highest F1-Score achieved by {table['F1 Score'].idxmax(axis = 0)} at = {max(table['F1 Score'] )}")
print(f"Highest Precision Score achieved by {table['Precision'].idxmax(axis = 0)} at = {max(table['Precision'] )}")
print(f"Highest Recall Score achieved by {table['Recall'].idxmax(axis = 0)} at = {max(table['Recall'] )}")

Highest Accuracy achieved by RF at = 98.59
Highest F1-Score achieved by RF at = 98.59
Highest Precision Score achieved by RF at = 98.59
Highest Recall Score achieved by RF at = 98.59
